In [2]:
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import os

/Users/ikerochoa/opt/anaconda3/envs/kaizen-copilot/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Initialize Chroma client
chroma_client = chromadb.HttpClient(host="localhost", port=8005, settings=Settings())

# Load sentence transformer (example model)
embedder = SentenceTransformer('all-MiniLM-L6-v2')

# Create or get collection
collection = chroma_client.get_collection(name="mwaskom_seaborn")

In [13]:
files_list = ["sample.py", "sample2.py", "sample3.py", "sample4.py", "sample5.py", "sample6.py"]
docs = []
embedds = []

for file_name in files_list:
    file_path = os.path.abspath(f"./sample_files/{file_name}")
    
    with open(file_path, "r", encoding="utf-8") as f:
        file_content = f.read()

    # Prepare text to embed (file name + content)
    text_to_embed = f"{file_path} {file_content}"
    docs.append(text_to_embed)

    embedds.append(embedder.encode([text_to_embed])[0].tolist())

In [14]:
import numpy
import torch

print("NumPy version:", numpy.__version__)
x = torch.tensor([1.0, 2.0, 3.0])
print("To NumPy:", x.numpy())  # <- Aquí debe funcionar sin errores

NumPy version: 1.26.4
To NumPy: [1. 2. 3.]


In [15]:
collection.upsert(
    ids=[files_list[i].split(".")[0] for i in range(len(docs))],  # unique ID
    documents=docs,
    embeddings=embedds,
    metadatas=[{"name": files_list[i], "extension": "py", "path": f"sample_files/{files_list[i]}"} for i in range(len(docs))]
)

In [16]:
results = collection.query(
    query_texts=["chroma_db/test/sample_files/sample2.py\ndef say_hi("], 
    n_results=2,
    where={"path": "sample_files/sample2.py"}
)

print(results)

{'ids': [['sample2']], 'distances': [[1.0407114]], 'embeddings': None, 'metadatas': [[{'name': 'sample2.py', 'path': 'sample_files/sample2.py', 'extension': 'py'}]], 'documents': [['/Users/ikerochoa/Desktop/Tec/8vo Sementre/kaizen-copilot/chroma_db/test/sample_files/sample2.py # ejemplo_avanzado.py\n\ndef dividir(a, b):\n    """Devuelve el resultado de dividir a entre b, manejando división por cero."""\n    try:\n        resultado = a / b\n    except ZeroDivisionError:\n        return "Error: No se puede dividir entre cero."\n    else:\n        return resultado\n\ndef say_hi(nombre, saludo="Hola"):\n    """Imprime un saludo personalizado, con saludo opcional."""\n    print(f"{saludo}, {nombre}!")\n\ndef promedio_lista(numeros):\n    """Calcula el promedio de una lista de números."""\n    if not numeros:\n        return 0\n    return sum(numeros) / len(numeros)\n\ndef contar_ocurrencias(lista):\n    """Devuelve un diccionario con el conteo de ocurrencias de cada elemento."""\n    conteo

Delete all collections:

In [ ]:
# from chromadb import HttpClient

# client = HttpClient(host="localhost", port=8005)
# collections = client.list_collections()

# for col in collections:
#     client.delete_collection(name=col.name)

# print("✅ Todas las colecciones fueron eliminadas.")

✅ Todas las colecciones fueron eliminadas.


## Revisar el embedding, que sea compatible para evitar perdida de contexto